In [1]:
import robot
import map
import solver
import utils

import numpy as np
import gtsam
import matplotlib.pyplot as plt

%matplotlib qt

# unitTest 1: Move robot and check odometrey measurements

In [2]:
car = robot.robot()

#control inputs
dr = 0.4 #[m]
dtheta = np.radians(20)

time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()

#set graphics
fig , ax = utils.setWorldMap()
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)
ax.legend(["ground truth","dead reckoning"])

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= timeFinal:
        meas_odom = car.moveAndMeasureOdometrey((dr,dtheta))
        
        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta #update angle
        xcurrent_DR[0] += meas_odom.dr * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dr * np.sin(xcurrent_DR[2])
        
        #log history
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_GT = np.vstack([hist_GT,car.pose])

        #increase time
        time += dt

        #plot
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(dt)

# unitTest 2: Build prior map

In [4]:
np.random.seed(seed=2)

N = 10 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)
sigMin = -0.5
sigMax = 0.5

landmarks = [None] * N
for ii in range(N):
    sigs = np.random.uniform(low=sigMin, high=sigMax, size=(2,2))
    cov = sigs@sigs.T #enforce symmetric and positive definite: https://mathworld.wolfram.com/PositiveDefiniteMatrix.html
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 cov = cov)

priorMap = map.map(landmarks)
priorMap.plot(plotIndex = True,plotCov = True)

# unitTest 3: Move car and check landmark measurements

In [7]:
#------Build worldmap
fig , ax = utils.setWorldMap()
np.random.seed(seed=0)

N = 10 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)

landmarks = [None] * N
for ii in range(N):
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 )
worldMap = map.map(landmarks)
worldMap.plot(ax = ax, plotIndex = True,plotCov = False)

#------Spawn Robot
car = robot.robot()
#control inputs
dr = 0.4 #[m]
dtheta = np.radians(20)
odom = (dr,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= timeFinal:
        meas_odom = car.moveAndMeasureOdometrey((dr,dtheta))

        meas_lm = car.measureLandmarks(worldMap)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta #update angle
        xcurrent_DR[0] += meas_odom.dr * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dr * np.sin(xcurrent_DR[2])
        
        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_LM.append(meas_lm)

        #increase time
        time += dt

        #plot
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(dt)

print(hist_LM)

[[], [], [<robot.meas_landmark object at 0x7f6a5f415fd0>], [<robot.meas_landmark object at 0x7f6a5f415940>], [<robot.meas_landmark object at 0x7f6a5f0588e0>, <robot.meas_landmark object at 0x7f6a5f02d7f0>], [<robot.meas_landmark object at 0x7f6a5f610eb0>, <robot.meas_landmark object at 0x7f6a5f04ba30>, <robot.meas_landmark object at 0x7f6a5f02d490>], [<robot.meas_landmark object at 0x7f6a5f415670>, <robot.meas_landmark object at 0x7f6a5f02d3a0>, <robot.meas_landmark object at 0x7f6a5f02d7c0>], [<robot.meas_landmark object at 0x7f6a5f02d880>, <robot.meas_landmark object at 0x7f6a5f04b790>], [<robot.meas_landmark object at 0x7f6a5f02d940>, <robot.meas_landmark object at 0x7f6a5f045e50>], [<robot.meas_landmark object at 0x7f6a5f04b7c0>], [], [], [], [], [], [], [], [], [], [], [<robot.meas_landmark object at 0x7f6a5f41aca0>]]


# unitTest4: Plot Robot Cone

In [6]:
#------Build worldmap
fig , ax = utils.setWorldMap()
np.random.seed(seed=0)

#------Spawn Robot
car = robot.robot()
car.plot(ax)
#control inputs
dr = 0.4 #[m]
dtheta = np.radians(20)
odom = (dr,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

with plt.ion():
    while time <= timeFinal:
        car.moveAndMeasureOdometrey((dr,dtheta))

        #increase time
        time += dt

        car.plot()
        plt.draw()
        plt.pause(dt)

# unitTest5: putting everything together before gtsam

In [2]:
car, worldMap, ax, fig = utils.default_world()
prms = utils.default_parameters()

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 1)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 1)

#run and plot simulation
time = 0
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= prms["time"]["timeFinal"]:
        meas_odom = car.moveAndMeasureOdometrey((prms["odom"]["dr"],prms["odom"]["dtheta"]))

        meas_lm = car.measureLandmarks(worldMap)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta #update angle
        xcurrent_DR[0] += meas_odom.dr * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dr * np.sin(xcurrent_DR[2])
        
        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_LM.append(meas_lm)

        #increase time
        time += prms["time"]["dt"]

        #plot
        car.plot()
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(prms["time"]["dt"])

print(hist_LM)

[[<robot.meas_landmark object at 0x7f4a70ce9f40>, <robot.meas_landmark object at 0x7f4a70cff070>, <robot.meas_landmark object at 0x7f4a70cff0d0>, <robot.meas_landmark object at 0x7f4a70cff130>, <robot.meas_landmark object at 0x7f4a70cff190>, <robot.meas_landmark object at 0x7f4a70cff1f0>, <robot.meas_landmark object at 0x7f4a70cff250>, <robot.meas_landmark object at 0x7f4a70cff2b0>], [<robot.meas_landmark object at 0x7f4a70ce9e80>, <robot.meas_landmark object at 0x7f4a70c6fa90>, <robot.meas_landmark object at 0x7f4a70c73880>, <robot.meas_landmark object at 0x7f4a70c738e0>, <robot.meas_landmark object at 0x7f4a70c73940>, <robot.meas_landmark object at 0x7f4a70c739a0>, <robot.meas_landmark object at 0x7f4a70c73a00>], [<robot.meas_landmark object at 0x7f4a70cff4c0>, <robot.meas_landmark object at 0x7f4a70c73a90>, <robot.meas_landmark object at 0x7f4a70c73b20>, <robot.meas_landmark object at 0x7f4a70c73cd0>, <robot.meas_landmark object at 0x7f4a70c73d60>, <robot.meas_landmark object at 0x7

# unitTest6 GTSAM

notes: </br>
examples folder: /home/alon18/LocalInstalls/miniconda3/envs/smallslam/lib/python3.9/site-packages/gtsam/examples </br>
* imuFactorISAM2Example.py - understand difference between graph.add and graph.push_back </br>
  + never mind, they are the same (?) https://gtsam.org/doxygen/a03480.html#a1bd7154e74538b5934fc616bb5ef7281 </br>
* PlanarSLAMExample.py- factor for range and bearing </br>


from VisualISAMExample.py: </br>
      # If this is the first iteration, add a prior on the first pose to set the coordinate frame </br>
        # and a prior on the first landmark to set the scale </br>
        # Also, as iSAM solves incrementally, we must wait until each is observed at least twice before </br>
        # adding it to iSAM. </br>

In [14]:
car, worldMap, ax, fig = utils.default_world()
prms = utils.default_parameters()
backend = solver.solver()

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []
hist_solverResults = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 1)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 1)

#run and plot simulation
time = 0
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= prms["time"]["timeFinal"]:
        meas_odom = car.moveAndMeasureOdometrey((prms["odom"]["dr"],prms["odom"]["dtheta"]))
        meas_lms = car.measureLandmarks(worldMap)

        backend.updateTimeIndex()
        backend.addOdomMeasurement(meas_odom)
        for meas_lm in meas_lms:
            backend.addlandmarkMeasurement(meas_lm)
        backend.updateGraph()

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta #update angle
        xcurrent_DR[0] += meas_odom.dr * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dr * np.sin(xcurrent_DR[2])
        
        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_LM.append(meas_lm)
        hist_solverResults.append(backend.getResult())

        #increase time
        time += prms["time"]["dt"]

        #plot
        car.plot()
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(prms["time"]["dt"])

TypeError: X(): incompatible function arguments. The following argument types are supported:
    1. (j: int) -> int

Invoked with: -1

In [10]:
n = gtsam.noiseModel.Base
n.print()

TypeError: print(): incompatible function arguments. The following argument types are supported:
    1. (self: gtsam.gtsam.noiseModel.Base, s: str = '') -> None

Invoked with: 